In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import zipfile
import csv
import cv2
import shutil
import random
import seaborn as sns
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.models import Model,Sequential
from keras.utils.vis_utils import plot_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications import vgg16


In [2]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

GPU device not found
Found GPU at: 


In [3]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [4]:
train = pd.read_csv('./LabelledDataSet/full_df.csv')
train.head(500)


,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,565,59,Female,565_left.jpg,565_right.jpg,dry age-related macular degeneration,dry age-related macular degeneration,0,0,0,0,1,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['A'],"[0, 0, 0, 0, 1, 0, 0, 0]",565_right.jpg
496,566,60,Male,566_left.jpg,566_right.jpg,drusen,moderate non proliferative retinopathy，drusen,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",566_right.jpg
497,567,60,Male,567_left.jpg,567_right.jpg,mild nonproliferative retinopathy,pathological myopia,0,1,0,0,0,0,1,0,../input/ocular-disease-recognition-odir5k/ODI...,['M'],"[0, 0, 0, 0, 0, 0, 1, 0]",567_right.jpg
498,569,64,Male,569_left.jpg,569_right.jpg,severe proliferative diabetic retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",569_right.jpg


# Visualizing the data

In [ ]:
def check_disease(df,start,end):
    df = df.iloc[:, start:end]
    disease_name, zeroCount, oneCount = [], [], []
    rowLen = len(df)
    for (column_name, column) in df.iteritems():
        disease_name.append(column_name)    
        zeroCount.append(df[column_name].value_counts()[0])
    oneCount = [rowLen - x for x in zeroCount]
    
    return disease_name, zeroCount, oneCount

In [ ]:
def check_disease(df,start,end):
    df = df.iloc[:, start:end]
    disease_name, zeroCount, oneCount = [], [], []
    rowLen = len(df)
    for (column_name, column) in df.iteritems():
        disease_name.append(column_name)    
        zeroCount.append(df[column_name].value_counts()[0])
    oneCount = [rowLen - x for x in zeroCount]
    
    return disease_name, zeroCount, oneCount

In [ ]:
disease_name, zeroCount, oneCount = check_disease(train,7,15)

fig = plt.figure(figsize=(15, 8))
plt.bar(disease_name, oneCount, width=0.6, color=['skyblue', 'gray'])
plt.xlabel('Disease Name')
plt.savefig('Overall Disease.png')

In [ ]:
# Drops the Column 'N', 
train=train.drop(columns=['N', 'D', 'H', 'O' ],axis=1)

#Drops the Rows
train=train[((train['G'] == 1) | (train['C'] == 1) | (train['A']== 1) | (train['M'] == 1))]

# Classes
classes=['G', 'C', 'A', 'M']

# Reset an Index of pandas dataFrame
train = train.reset_index(drop=True)

In [ ]:
disease_name, zeroCount, oneCount = check_disease(train,7,11)

fig = plt.figure(figsize=(10, 8))
plt.bar(disease_name, oneCount, width=0.6, color=['skyblue', 'gray'])
plt.xlabel('Disease Name')
plt.savefig('Final Disease.png')

In [ ]:
def plot_count(feature, title, df, size=1, show_all=False):
    f, ax = plt.subplots(1,1, figsize=(4*size,4))
    total = float(len(df))
    if show_all:
        g = sns.countplot(df[feature], palette='Set3')
        g.set_title("{} distribution".format(title))
    else:
        g = sns.countplot(df[feature], order = df[feature].value_counts().index[:20], palette='Set3')
        if(size > 2):
            plt.xticks(rotation=90, size=8)
            for p in ax.patches:
                height = p.get_height()
                ax.text(p.get_x()+p.get_width()/2.,
                        height + 0.2,
                        '{:1.2f}%'.format(100*height/total),
                        ha="center") 
        g.set_title("Number and percentage of {}".format(title))
    plt.show() 

In [ ]:
plot_count("Patient Age", "Age", train, size=5, show_all=True)

In [ ]:
plot_count("Patient Sex", "Sex", train, size=2)

In [ ]:
plot_count("G", "Glaucoma", train, size=2)

In [ ]:
plot_count("C", "Cataract", train, size=2)

In [ ]:
plot_count("A", "Age Related Macular Degeneration", train, size=2)

In [ ]:
plot_count("M", "Pathological Myopia", train, size=2)

In [ ]:
def plot_feature_distribution_grouped(feature, title, df, hue, size=4):
    plt.figure(figsize=(size*5,size*2))
    plt.title(title)
    if(size > 2):
        plt.xticks(rotation=90, size=8)
    g = sns.countplot(df[feature], hue=df[hue], palette='Set3')
    plt.xlabel(feature)
    plt.legend()
    plt.show()

In [ ]:
plot_feature_distribution_grouped('Patient Sex', 'Glaucoma diagnosys grouped by sex', train, 'G', size=2)

In [ ]:
plot_feature_distribution_grouped('Patient Sex', 'Cataract diagnosys grouped by sex', train, 'C', size=2)

In [ ]:
plot_feature_distribution_grouped('Patient Sex', 'AMD diagnosys grouped by sex', train, 'A', size=2)

In [ ]:
plot_feature_distribution_grouped('Patient Sex', 'Pathological Myopia diagnosys grouped by sex', train, 'M', size=2)

# Pre processing the data

In [ ]:
# Create List for each Disease
g_disease=[]
for i in range(len(train)):
    if(train['G'][i]==1):
        g_disease.append(train['filename'][i])
        
c_disease=[]
for i in range(len(train)):
    if(train['C'][i]==1):
        c_disease.append(train['filename'][i])

a_disease=[]
for i in range(len(train)):
    if(train['A'][i]==1):
        a_disease.append(train['filename'][i])
        
m_disease=[]
for i in range(len(train)):
    if(train['M'][i]==1):
        m_disease.append(train['filename'][i])

In [ ]:
# Copy images to new path
if os.path.isdir('./data') is False:
    os.mkdir('./data')

# (Glaucoma, Cataract, Age related Macular Degeneration, Hypertension)

g_disease_save_dir='./data/glaucoma/'
os.mkdir('./data/glaucoma/')
for filename in tqdm(g_disease):
  shutil.copy('./LabelledDataSet/preprocessed_images/'+filename,g_disease_save_dir)

c_disease_save_dir='./data/cataract/'
os.mkdir('./data/cataract/')
for filename in tqdm(c_disease):
  shutil.copy('./LabelledDataSet/preprocessed_images/'+filename,c_disease_save_dir)

a_disease_save_dir='./data/age_related_macular_degeneration/'
os.mkdir('./data/age_related_macular_degeneration/')
for filename in tqdm(a_disease):
  shutil.copy('./LabelledDataSet/preprocessed_images/'+filename,a_disease_save_dir)

m_disease_save_dir='./data/pathological_myopia/'
os.mkdir('./data/pathological_myopia/')
for filename in tqdm(m_disease):
  shutil.copy('./LabelledDataSet/preprocessed_images/'+filename,m_disease_save_dir)

In [ ]:
base_dir = './dataset/'
if os.path.isdir(base_dir) is False:
    os.mkdir(base_dir)

#create two folders (train and validation)
train_dir = os.path.join(base_dir, 'Training')
os.mkdir(train_dir)

testing_dir = os.path.join(base_dir, 'Testing')
os.mkdir(testing_dir)

validation_dir = os.path.join(base_dir, 'Validation')
os.mkdir(validation_dir)

# Under train folder create four folders 
# (Glaucoma, Cataract, Age related Macular Degeneration, Pathological Myopia)

train_glaucoma_dir = os.path.join(train_dir, 'glaucoma')
os.mkdir(train_glaucoma_dir)

train_cataract_dir = os.path.join(train_dir, 'cataract')
os.mkdir(train_cataract_dir)

train_age_macular_dir = os.path.join(train_dir, 'age_related_macular_degeneration')
os.mkdir(train_age_macular_dir)

train_pathological_myopia_dir = os.path.join(train_dir, 'pathological_myopia')
os.mkdir(train_pathological_myopia_dir)


# Under testing folder create four folders 

testing_glaucoma_dir = os.path.join(testing_dir, 'glaucoma')
os.mkdir(testing_glaucoma_dir)

testing_cataract_dir = os.path.join(testing_dir, 'cataract')
os.mkdir(testing_cataract_dir)

testing_age_macular_dir = os.path.join(testing_dir, 'age_related_macular_degeneration')
os.mkdir(testing_age_macular_dir)

testing_pathological_myopia_dir = os.path.join(testing_dir, 'pathological_myopia')
os.mkdir(testing_pathological_myopia_dir)


# Under validation folder create four folders 
# (Glaucoma, Cataract, Age related Macular Degeneration, Pathological Myopia)

validation_glaucoma_dir = os.path.join(validation_dir, 'glaucoma')
os.mkdir(validation_glaucoma_dir)

validation_cataract_dir = os.path.join(validation_dir, 'cataract')
os.mkdir(validation_cataract_dir)

validation_age_macular_dir = os.path.join(validation_dir, 'age_related_macular_degeneration')
os.mkdir(validation_age_macular_dir)

validation_pathological_myopia_dir = os.path.join(validation_dir, 'pathological_myopia')
os.mkdir(validation_pathological_myopia_dir)

In [ ]:
glaucoma_source_dir = './data/glaucoma/'
training_glaucoma_dir = './dataset/Training/glaucoma/'
testing_glaucoma_dir = './dataset/Testing/glaucoma/'
valid_glaucoma_dir = './dataset/Validation/glaucoma/'

cataract_source_dir = './data/cataract/'
training_cataract_dir = './dataset/Training/cataract/'
testing_cataract_dir = './dataset/Testing/cataract/'
valid_cataract_dir = './dataset/Validation/cataract/'

age_related_macular_degeneration_source_dir = './data/age_related_macular_degeneration/'
training_age_related_macular_degeneration_dir = './dataset/Training/age_related_macular_degeneration/'
testing_age_related_macular_degeneration_dir = './dataset/Testing/age_related_macular_degeneration/'
valid_age_related_macular_degeneration_dir = './dataset/Validation/age_related_macular_degeneration/'

pathological_myopia_source_dir = './data/pathological_myopia/'
training_pathological_myopia_dir = './dataset/Training/pathological_myopia/'
testing_pathological_myopia_dir = './dataset/Testing/pathological_myopia/'
valid_pathological_myopia_dir = './dataset/Validation/pathological_myopia/'


In [ ]:
def split_data(SOURCE, TRAINING, VALIDATION, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")
    
    SIZE = (1 - SPLIT_SIZE) / 2
    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) * SIZE)
    valid_length = int(len(files) * SIZE)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[training_length:training_length+testing_length]
    valid_set = shuffled_set[training_length+testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        shutil.copy(this_file, destination)

    for filename in valid_set:
        this_file = SOURCE + filename
        destination = VALIDATION + filename
        shutil.copy(this_file, destination)
        
    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        shutil.copy(this_file, destination)

In [ ]:
split_size = 0.80

split_data(glaucoma_source_dir, training_glaucoma_dir, testing_glaucoma_dir, valid_glaucoma_dir, split_size)
split_data(cataract_source_dir, training_cataract_dir, testing_cataract_dir, valid_cataract_dir, split_size)
split_data(age_related_macular_degeneration_source_dir, training_age_related_macular_degeneration_dir, testing_age_related_macular_degeneration_dir, valid_age_related_macular_degeneration_dir, split_size)
split_data(pathological_myopia_source_dir, training_pathological_myopia_dir, testing_pathological_myopia_dir, valid_pathological_myopia_dir, split_size)

In [ ]:
def img_p(img_height,img_width):
    train_datagen = ImageDataGenerator(rescale = 1./255.,
                                        rotation_range = 30,
                                        width_shift_range = 0.1,
                                        height_shift_range = 0.1,
                                        zoom_range = 0.1,
                                        horizontal_flip = True
                                       )
    train_generator = train_datagen.flow_from_directory('./dataset/Training/', 
                                                        target_size=(img_height,img_width), 
                                                        class_mode='categorical')
    validation_datagen = ImageDataGenerator(rescale=1/255)
    validation_generator = validation_datagen.flow_from_directory('./dataset/Validation/', 
                                                              target_size=(img_height,img_width),
                                                              class_mode='categorical')
    test_datagen = ImageDataGenerator(rescale=1/255)
    test_generator = test_datagen.flow_from_directory('./dataset/Testing/', 
                                                              target_size=(img_height,img_width),
                                                              class_mode='categorical')
    
    return train_generator,validation_generator,test_generator

# Model InceptionV3 

In [ ]:
train_generator,validation_generator,test_generator = img_p(299,299)

In [ ]:
base_model = InceptionV3(input_shape=(299, 299, 3), weights='imagenet', include_top=True)
base_model.summary()

In [ ]:
x = base_model.layers[-2].output
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense (4, activation='softmax')(x)

In [ ]:
base_m = Model(inputs=base_model.input, outputs=x)

In [ ]:
base_m.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy', 
                        tf.keras.metrics.Recall(),
                        tf.keras.metrics.Precision(),   
                        tf.keras.metrics.AUC(),
                        tfa.metrics.F1Score(num_classes=4, average="macro")])

In [ ]:
clf=base_m.fit(train_generator,
            validation_data = validation_generator,
            steps_per_epoch=10,
            epochs =10,
            validation_steps=5,
            verbose = 1)

In [ ]:
base_m.evaluate(test_generator)

In [ ]:
acc = clf.history['accuracy']
val_acc = clf.history['val_accuracy']
loss = clf.history['loss']
val_loss = clf.history['val_loss']

epochs = range(1, len(loss) + 1)

#accuracy plot
plt.figure(figsize=(10,5))
plt.plot(epochs, acc,  label='Training Accuracy') #color='lightblue',
plt.plot(epochs, val_acc,  label='Validation Accuracy') #color='orange',
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc="best")
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.savefig('inception_accuracy.png')

#loss plot
plt.figure(figsize=(10,5))
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.rcParams['axes.facecolor'] = 'white'
plt.savefig('inception_loss.png')


In [ ]:
base_m.predict(test_generator)

In [ ]:
m = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

In [ ]:
features = m.predict(train_generator)

In [ ]:
features.shape

In [ ]:
t_features = m.predict(test_generator)

In [ ]:
t_features.shape

In [ ]:
from numpy import save
save('train_fe',features)
save('test_fe',t_features)

In [ ]:
local_weights_file = './LabelledDataSet/inception/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(input_shape = (299, 299, 3), 
                                include_top = False, 
                                weights = None)
pre_trained_model.load_weights(local_weights_file)
for layer in pre_trained_model.layers:
  layer.trainable = False
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
x = layers.Flatten()(last_output)
x = layers.Dropout(0.5)(x) 
x = layers.Dense(1024, activation='relu')(x)                 
x = layers.Dense (4, activation='softmax')(x)           
model = Model( pre_trained_model.input, x) 
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch=10,
            epochs =10,
            validation_steps=5,
            verbose = 1)

In [ ]:
model.evaluate(test_generator)

In [ ]:
model.save("inception_v3.h5")

In [ ]:
#plotting training values
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

#accuracy plot
plt.figure(figsize=(10,5))
plt.plot(epochs, acc,  label='Training Accuracy') #color='lightblue',
plt.plot(epochs, val_acc,  label='Validation Accuracy') #color='orange',
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc="best")
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.savefig('inception_weights_accuracy.png')

#loss plot
plt.figure(figsize=(10,5))
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.rcParams['axes.facecolor'] = 'white'
plt.savefig('inception_weights_loss.png')

# Model ResNet

In [ ]:
train_generator,validation_generator,test_generator = img_p(224,224)

In [ ]:
resnet_m = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=True)

In [ ]:
resnet_m.summary()

In [ ]:
x = resnet_m.layers[-2].output
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense (4, activation='softmax')(x)

In [ ]:
resnet_m = Model(inputs=resnet_m.input, outputs=x)
resnet_m.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
res_clf = resnet_m.fit(train_generator,
            validation_data = validation_generator,
            steps_per_epoch=10,
            epochs =10,
            validation_steps=5,
            verbose = 1)

In [ ]:
#plotting training values
import matplotlib.pyplot as plt

acc = res_clf.history['accuracy']
val_acc = res_clf.history['val_accuracy']
loss = res_clf.history['loss']
val_loss = res_clf.history['val_loss']

epochs = range(1, len(loss) + 1)

#accuracy plot
plt.figure(figsize=(10,5))
plt.plot(epochs, acc,  label='Training Accuracy') #color='lightblue',
plt.plot(epochs, val_acc,  label='Validation Accuracy') #color='orange',
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc="best")
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.savefig('resnet_accuracy.png')

#loss plot
plt.figure(figsize=(10,5))
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.rcParams['axes.facecolor'] = 'white'
plt.savefig('resnet_loss.png')

In [ ]:
pre_trained_model = ResNet50(input_shape = (224,224, 3),
                        include_top = False,
                        weights = 'imagenet')

for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
resNet_model = Sequential([
        pre_trained_model,
        Flatten(),
        Dropout(0.5),
        Dense(1024, activation='relu'),
        Dense(4, activation='softmax')])

In [ ]:
resNet_model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
input_shape = (None, 224, 224, 3)
resNet_model.build(input_shape)

In [ ]:
resNet_model.summary()

In [ ]:
history = resNet_model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch=10,
            epochs =10,
            validation_steps=5,
            verbose = 1)

In [ ]:
resNet_model.evaluate(test_generator)

In [ ]:
#plotting training values
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

#accuracy plot
plt.figure(figsize=(10,5))
plt.plot(epochs, acc,  label='Training Accuracy') #color='lightblue',
plt.plot(epochs, val_acc,  label='Validation Accuracy') #color='orange',
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc="best")
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.savefig('resnet_weights_accuracy.png')

#loss plot
plt.figure(figsize=(10,5))
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.rcParams['axes.facecolor'] = 'white'
plt.savefig('resnet_weights_loss.png')

In [ ]:
resNet_model.save("resnet_model.h5")

# Model VGG16

In [ ]:
model=vgg16.VGG16(weights='imagenet',input_shape=(224,224,3), include_top=True, pooling="avg")

In [ ]:
model.summary()

In [ ]:
x = model.layers[-2].output
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense (4, activation='softmax')(x)

In [ ]:
b_mvgg16 = Model(inputs=model.input, outputs=x)
b_mvgg16.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])



In [ ]:
vgg_clf = b_mvgg16.fit(train_generator,
            validation_data = validation_generator,
            steps_per_epoch=10,
            epochs =10,
            validation_steps=5,
            verbose = 1)

In [ ]:
b_mvgg16.evaluate(test_generator)

In [ ]:
#plotting training values
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

#accuracy plot
plt.figure(figsize=(10,5))
plt.plot(epochs, acc,  label='Training Accuracy') #color='lightblue',
plt.plot(epochs, val_acc,  label='Validation Accuracy') #color='orange',
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc="best")
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.savefig('vgg_accuracy.png')

#loss plot
plt.figure(figsize=(10,5))
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.rcParams['axes.facecolor'] = 'white'
plt.savefig('vgg_loss.png')

In [ ]:
model_vgg = vgg16.VGG16(weights='imagenet', input_shape=(224,224,3), include_top=False)
model_vgg.summary()
  # Make all layers untrainable
for layer in model_vgg.layers:
    layer.trainable = False

In [ ]:
x = model_vgg.output
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(4, activation="softmax")(x)

In [ ]:
m = Model(inputs=model_vgg.input, outputs=predictions)

In [ ]:
m.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
vgg_clfw = m.fit(train_generator,
                epochs=1,
                steps_per_epoch=10,
                validation_data=validation_generator,
                validation_steps=5)

In [ ]:
m.evaluate(test_generator)

In [ ]:
#plotting training values
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

#accuracy plot
plt.figure(figsize=(10,5))
plt.plot(epochs, acc,  label='Training Accuracy') #color='lightblue',
plt.plot(epochs, val_acc,  label='Validation Accuracy') #color='orange',
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc="best")
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.savefig('vgg_weights_accuracy.png')

#loss plot
plt.figure(figsize=(10,5))
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc="best")
plt.grid(visible=True, which='major', color='#666666', linestyle='-')
plt.tight_layout()
plt.rcParams['axes.facecolor'] = 'white'
plt.savefig('vgg_weights_loss.png')

In [ ]:
m.save("vgg_model.h5")